In [ ]:
import json
import pandas as pd
import dill as pickle

class DataUtil2:
    """"""
    @classmethod
    def load(cls, file, format, date_cols= None):
        """"""
        data = ''
        if(format== 'csv'):
            data = pd.read_csv(file, parse_dates= date_cols)
        elif(format== 'json'):
            with open(file, 'r') as i_file:
                data = json.load(file)
            i_file.close()
        elif(format== 'pkl'):
            with open(file, 'rb') as i_file:
                data = pickle.load(i_file)
            i_file.close()
        elif(format == 'hdf'):
            data = pd.read_hdf(path_or_buf= file, key='undefined')

        return  data

    @classmethod
    def save(cls, data, file, format, precision= 8):
        """"""
        if(format == 'csv'):
            data.to_csv(file, float_format= '%%.%df' % precision, index= False)
        elif(format == 'json'):
            with open(file, 'w') as o_file:
                json.dump(data, o_file, ensure_ascii= True, indent= 4)
            o_file.close()
        elif(format == 'pkl'):
            with open(file, 'wb') as o_file:
                pickle.dump(data, o_file, -1)
            o_file.close()
        elif(format== 'hdf'):
            data.to_hdf(path_or_buf= file, key='undefined', mode='w', complib='blosc')

        return

In [ ]:
#######################
# LigthGBM Regression #
#######################
import numpy as np
import pandas as pd
import datetime
import time
import os,sys
import gc
from sklearn import *
import lightgbm
import random
import json

drop_cols = ['id', 'visit_date', 'visitors', 'hpg_store_id', 'fold', 'air_store_id']

cate_cols = ['store_id_encoded', 'area_name', 'city', 'genre_name']
cate_feats = ['dow', 'hol_days', 'day', 'pom', 'prev_is_holiday', 'next_is_holiday', 
              'wom', 'woy', 'is_weekends', 'holiday_flg', 'month', 'is_up_corner']
for mod in ['air', 'hpg']:
    cate_feats.extend(['%s_%s' % (mod, c) for c in cate_cols])
    
def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5

DataBaseDir = '../../data'
InputDir = '%s/l0/kfold' % DataBaseDir
kfold = 5
strategy = 'lgb_l2'
#### load data
valid_dfs = []
holdout_dfs = []
test_dfs = []
trees = ['NN_EF']
for fold in range(kfold):
    FoldInputDir = '%s/%s' % (InputDir, fold)
    valid = pd.read_csv('%s/valid.csv' % FoldInputDir, parse_dates= ['visit_date']).reset_index(drop= True)
    holdout = pd.read_csv('%s/holdout.csv' % FoldInputDir, parse_dates= ['visit_date']).reset_index(drop= True)
    test = pd.read_csv('%s/test.csv' % FoldInputDir, parse_dates= ['visit_date']).reset_index(drop= True)
    for t in trees:
        # load cb_ef
        FoldOutputDir = '%s/l1/kfold/%s' % (DataBaseDir, fold)
        valid_cb_ef = pd.read_csv('%s/valid_%s.csv' % (FoldOutputDir, t), parse_dates= ['visit_date']).reset_index(drop= True)
        holdout_cb_ef = pd.read_csv('%s/holdout_%s.csv' % (FoldOutputDir, t), parse_dates= ['visit_date']).reset_index(drop= True)
        test_cb_ef = pd.read_csv('%s/test_%s.csv' % (FoldOutputDir, t), parse_dates= ['visit_date']).reset_index(drop= True)
        # concate
        valid = pd.concat([valid, valid_cb_ef[[t]]], axis= 1)
        holdout = pd.concat([holdout, holdout_cb_ef[[t]]], axis= 1)
        test = pd.concat([test, test_cb_ef[[t]]], axis= 1)
    #
    valid['fold'] = fold
    valid_dfs.append(valid)
    holdout_dfs.append(holdout)
    test_dfs.append(test)
TrainData = pd.concat(valid_dfs, axis= 0, ignore_index= True)
##### model selection with CV
# score
holdout_score = .0
# parameters
params = {
    "boosting": "gbdt",
    "objective": "regression_l2",
    "lambda_l2": [200, 20, 1],
    "learning_rate": [0.4, 0.2, 0.1],
    "max_depth": [8, 12, 16],
    
#     "lambda_l2": [20],
#     "learning_rate": [0.2],
#     "max_depth": [16],
#     "device": "gpu",
    
    "num_iterations": 150,
    "min_data_in_leaf": 20,
    'num_leaves': 255,

    "feature_fraction": 0.90,
    "bagging_fraction": 0.85,
    "bagging_freq": 20,
    "min_hessian": 0.001,

    "max_bin": 63,
}
total_cols = [c for c in TrainData.columns if c not in drop_cols]
K = int(0.3 * len(total_cols))
N = 200

start = time.time()
for idx in range(N): 
    selected_cols = [total_cols[i] for i in sorted(random.sample(range(len(total_cols)), K))]
    OutputDir = '%s/MM/l1/%s' % (DataBaseDir, idx)
    if(os.path.exists(OutputDir) == False):
        os.makedirs(OutputDir)
    # save feature space
    with open('%s/sub_feats.txt' % OutputDir, 'w') as o_file:
        for feat in selected_cols:
            o_file.write('%s\n' % feat)
    o_file.close()
    ##
    BestParmas = {}
    BestScore = 1.0
    cv_score = .0
    for l2 in params['lambda_l2']:
        for lr in params['learning_rate']:
            for depth in params['max_depth']:
                cv_rmsle = .0
                for fold in range(kfold):
                    FoldData = {
                        'train': TrainData[TrainData['fold'] != fold],
                        'valid': TrainData[TrainData['fold'] == fold]
                    }
                    d_cv = lightgbm.Dataset(FoldData['train'][selected_cols], 
                                        label= FoldData['train']['visitors'], 
                                        #max_bin= params['max_bin'], 
                                        silent= True, 
                                        free_raw_data= True)
                    param = {
                        'boosting': 'gbdt',
                        'objective': 'regression_l2',

                        'lambda_l2': l2,
                        'learning_rate': lr,
                        'max_depth': depth,

                        'num_iterations': params['num_iterations'],
                        'feature_fraction': params['feature_fraction'],
                        'bagging_fraction': params['bagging_fraction'],
                        'bagging_freq': params['bagging_freq'],
                        'min_hessian': params['min_hessian'],
                        'max_bin': params['max_bin'],
                    }
                    model = lightgbm.train(param, d_cv)
                    FoldData['valid'][strategy] = model.predict(FoldData['valid'][selected_cols])
                    rmsle_valid = RMSLE(FoldData['valid']['visitors'], FoldData['valid'][strategy])
                    cv_rmsle += rmsle_valid
                cv_rmsle /= kfold
                if(cv_rmsle < BestScore):
                    BestScore = cv_rmsle
                    BestParmas['lambda_l2'] = l2
                    BestParmas['learning_rate'] = lr
                    BestParmas['max_depth'] = depth
                end = time.time()
                print('running for params, l2 %s, lr %s, depth %s done, cv score %.5f, time elapsed %.2fs' % (l2, lr, depth, cv_rmsle, (end - start)))
    end = time.time()
    print('grid search done, time elapsed %.2fs' % (end - start))
    print('best params, %s' % BestParmas)
    ## retrain and store
    cv_score = .0
    holdout_score = .0
    for fold in range(kfold):
        FoldData = {
            'train': TrainData[TrainData['fold'] != fold],
            'valid': TrainData[TrainData['fold'] == fold],
            'holdout': holdout_dfs[fold],
            'test': test_dfs[fold]
        }
        # train
        d_cv = lightgbm.Dataset(FoldData['train'][selected_cols], 
                            label= FoldData['train']['visitors'].values, 
                            #max_bin= params['max_bin'], 
                            silent= True, 
                            free_raw_data= True)
        param = {
            'boosting': 'gbdt',
            'objective': 'regression_l2',
                
            'lambda_l2': BestParmas['lambda_l2'],
            'learning_rate': BestParmas['learning_rate'],
            'max_depth': BestParmas['max_depth'],
                        
            'num_iterations': params['num_iterations'],
            'feature_fraction': params['feature_fraction'],
            'bagging_fraction': params['bagging_fraction'],
            'bagging_freq': params['bagging_freq'],
            'min_hessian': params['min_hessian'],
            'max_bin': params['max_bin'],
        }
        model = lightgbm.train(param, d_cv)
        # for valid
        FoldData['valid'][strategy] = model.predict(FoldData['valid'][selected_cols])
        rmsle_valid = RMSLE(FoldData['valid']['visitors'].values, FoldData['valid'][strategy])
        cv_score += rmsle_valid
        # for holdout
        FoldData['holdout'][strategy] = model.predict(FoldData['holdout'][selected_cols])
        rmsle_holdout = RMSLE(FoldData['holdout']['visitors'].values, FoldData['holdout'][strategy])
        holdout_score += rmsle_holdout
        # for test
        FoldData['test'][strategy] = model.predict(FoldData['test'][selected_cols])

        print('fold %s: valid score %.6f, holdout score %.6f, valid length %s' % (fold, rmsle_valid, rmsle_holdout, len(FoldData['valid'])))  
        #### output
        FoldOutputDir = '%s/kfold/%s' % (OutputDir, fold)
        if(os.path.exists(FoldOutputDir) == False):
            os.makedirs(FoldOutputDir)
        for mod in FoldData.keys():
            if(mod == 'train'):
                continue
            OutCols = []
            if(mod == 'test'):
                OutCols.append('id')
            OutCols.extend(['air_store_id', 'visit_date', 'visitors', strategy])
            OutputFile = '%s/%s_%s.csv' % (FoldOutputDir, mod, strategy)
            OutFoldData = FoldData[mod][OutCols]
            OutFoldData.to_csv(OutputFile, index= False)
    
    cv_score /= kfold # Average valid set predictions
    holdout_score /= kfold # Average holdout set predictions

    end = time.time()
    print('\n======================')
    print("CV score %.4f, Holdout score %.4f, Elapsed time: %.2fs" % (cv_score, holdout_score, (end - start)))
    print('======================\n')
    
    with open('%s/result.txt' % OutputDir, 'w') as o_file:
        o_file.write('%s\n' % json.dumps(BestParmas))
        o_file.write('cv score %.4f, holdout score %.4f' % (cv_score, holdout_score))
    o_file.close()
    
    print('------- part %s done. time elapsed %.2fs ----------\n' % (idx, (end - start)))
#     if(idx == 0):
#         break

/home/joe/.local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/joe/.local/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/home/joe/.local/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarn

running for params, l2 200, lr 0.4, depth 8 done, cv score 0.50599, time elapsed 4.05s
running for params, l2 200, lr 0.4, depth 12 done, cv score 0.50608, time elapsed 7.68s
running for params, l2 200, lr 0.4, depth 16 done, cv score 0.50610, time elapsed 11.30s
running for params, l2 200, lr 0.2, depth 8 done, cv score 0.50455, time elapsed 15.08s
running for params, l2 200, lr 0.2, depth 12 done, cv score 0.50455, time elapsed 18.90s
running for params, l2 200, lr 0.2, depth 16 done, cv score 0.50458, time elapsed 22.68s
running for params, l2 200, lr 0.1, depth 8 done, cv score 0.50410, time elapsed 26.83s
running for params, l2 200, lr 0.1, depth 12 done, cv score 0.50418, time elapsed 31.03s
running for params, l2 200, lr 0.1, depth 16 done, cv score 0.50423, time elapsed 34.88s
running for params, l2 20, lr 0.4, depth 8 done, cv score 0.50743, time elapsed 37.93s
running for params, l2 20, lr 0.4, depth 12 done, cv score 0.50730, time elapsed 40.99s
running for params, l2 20, lr

/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


fold 0: valid score 0.508477, holdout score 0.498542, valid length 45371
fold 1: valid score 0.499864, holdout score 0.499206, valid length 45371
fold 2: valid score 0.502655, holdout score 0.499247, valid length 45371
fold 3: valid score 0.506338, holdout score 0.498228, valid length 45370
fold 4: valid score 0.500887, holdout score 0.498328, valid length 45370

CV score 0.5036, Holdout score 0.4987, Elapsed time: 108.40s

------- part 0 done. time elapsed 108.40s ----------

running for params, l2 200, lr 0.4, depth 8 done, cv score 0.50147, time elapsed 112.03s
running for params, l2 200, lr 0.4, depth 12 done, cv score 0.50168, time elapsed 115.46s
running for params, l2 200, lr 0.4, depth 16 done, cv score 0.50180, time elapsed 119.05s
running for params, l2 200, lr 0.2, depth 8 done, cv score 0.50039, time elapsed 122.79s
running for params, l2 200, lr 0.2, depth 12 done, cv score 0.50035, time elapsed 126.41s
running for params, l2 200, lr 0.2, depth 16 done, cv score 0.50025, t

running for params, l2 1, lr 0.4, depth 8 done, cv score 0.52227, time elapsed 395.53s
running for params, l2 1, lr 0.4, depth 12 done, cv score 0.52197, time elapsed 398.88s
running for params, l2 1, lr 0.4, depth 16 done, cv score 0.52213, time elapsed 402.17s
running for params, l2 1, lr 0.2, depth 8 done, cv score 0.52043, time elapsed 405.88s
running for params, l2 1, lr 0.2, depth 12 done, cv score 0.52012, time elapsed 409.40s
running for params, l2 1, lr 0.2, depth 16 done, cv score 0.51988, time elapsed 412.98s
running for params, l2 1, lr 0.1, depth 8 done, cv score 0.52233, time elapsed 417.03s
running for params, l2 1, lr 0.1, depth 12 done, cv score 0.52216, time elapsed 421.11s
running for params, l2 1, lr 0.1, depth 16 done, cv score 0.52203, time elapsed 425.15s
grid search done, time elapsed 425.15s
best params, {'lambda_l2': 200, 'learning_rate': 0.4, 'max_depth': 12}
fold 0: valid score 0.522042, holdout score 0.515533, valid length 45371
fold 1: valid score 0.515592

...

running for params, l2 1, lr 0.4, depth 8 done, cv score 0.50667, time elapsed 6947.68s
running for params, l2 1, lr 0.4, depth 12 done, cv score 0.50675, time elapsed 6950.77s
running for params, l2 1, lr 0.4, depth 16 done, cv score 0.50689, time elapsed 6953.98s
running for params, l2 1, lr 0.2, depth 8 done, cv score 0.50215, time elapsed 6957.36s
running for params, l2 1, lr 0.2, depth 12 done, cv score 0.50245, time elapsed 6960.79s
running for params, l2 1, lr 0.2, depth 16 done, cv score 0.50211, time elapsed 6964.08s
running for params, l2 1, lr 0.1, depth 8 done, cv score 0.50147, time elapsed 6967.89s
running for params, l2 1, lr 0.1, depth 12 done, cv score 0.50136, time elapsed 6971.62s
running for params, l2 1, lr 0.1, depth 16 done, cv score 0.50131, time elapsed 6975.28s
grid search done, time elapsed 6975.28s
best params, {'lambda_l2': 20, 'learning_rate': 0.1, 'max_depth': 12}
fold 0: valid score 0.506305, holdout score 0.495757, valid length 45371
fold 1: valid score

running for params, l2 200, lr 0.1, depth 12 done, cv score 0.53156, time elapsed 7234.67s
running for params, l2 200, lr 0.1, depth 16 done, cv score 0.53160, time elapsed 7239.17s
running for params, l2 20, lr 0.4, depth 8 done, cv score 0.52897, time elapsed 7242.79s
running for params, l2 20, lr 0.4, depth 12 done, cv score 0.52851, time elapsed 7246.23s
running for params, l2 20, lr 0.4, depth 16 done, cv score 0.52881, time elapsed 7249.75s
running for params, l2 20, lr 0.2, depth 8 done, cv score 0.52859, time elapsed 7253.60s
running for params, l2 20, lr 0.2, depth 12 done, cv score 0.52816, time elapsed 7257.30s
running for params, l2 20, lr 0.2, depth 16 done, cv score 0.52819, time elapsed 7261.07s
running for params, l2 20, lr 0.1, depth 8 done, cv score 0.53101, time elapsed 7265.31s
running for params, l2 20, lr 0.1, depth 12 done, cv score 0.53049, time elapsed 7269.35s
running for params, l2 20, lr 0.1, depth 16 done, cv score 0.53068, time elapsed 7273.41s
running for

fold 3: valid score 0.527253, holdout score 0.517072, valid length 45370
fold 4: valid score 0.520957, holdout score 0.518806, valid length 45370

CV score 0.5234, Holdout score 0.5179, Elapsed time: 7527.04s

------- part 70 done. time elapsed 7527.04s ----------

running for params, l2 200, lr 0.4, depth 8 done, cv score 0.49506, time elapsed 7530.51s
running for params, l2 200, lr 0.4, depth 12 done, cv score 0.49487, time elapsed 7534.01s
running for params, l2 200, lr 0.4, depth 16 done, cv score 0.49495, time elapsed 7537.52s
running for params, l2 200, lr 0.2, depth 8 done, cv score 0.49374, time elapsed 7541.17s
running for params, l2 200, lr 0.2, depth 12 done, cv score 0.49371, time elapsed 7544.89s
running for params, l2 200, lr 0.2, depth 16 done, cv score 0.49379, time elapsed 7548.54s
running for params, l2 200, lr 0.1, depth 8 done, cv score 0.49406, time elapsed 7552.48s
running for params, l2 200, lr 0.1, depth 12 done, cv score 0.49405, time elapsed 7556.50s
running f

running for params, l2 1, lr 0.4, depth 16 done, cv score 0.51062, time elapsed 7808.36s
running for params, l2 1, lr 0.2, depth 8 done, cv score 0.50846, time elapsed 7811.68s
running for params, l2 1, lr 0.2, depth 12 done, cv score 0.50807, time elapsed 7814.99s
running for params, l2 1, lr 0.2, depth 16 done, cv score 0.50817, time elapsed 7818.36s
running for params, l2 1, lr 0.1, depth 8 done, cv score 0.50974, time elapsed 7822.05s
running for params, l2 1, lr 0.1, depth 12 done, cv score 0.50936, time elapsed 7825.73s
running for params, l2 1, lr 0.1, depth 16 done, cv score 0.50940, time elapsed 7829.42s
grid search done, time elapsed 7829.42s
best params, {'lambda_l2': 200, 'learning_rate': 0.4, 'max_depth': 16}
fold 0: valid score 0.511276, holdout score 0.501373, valid length 45371
fold 1: valid score 0.501979, holdout score 0.499932, valid length 45371
fold 2: valid score 0.507886, holdout score 0.500866, valid length 45371
fold 3: valid score 0.511448, holdout score 0.501

running for params, l2 20, lr 0.4, depth 8 done, cv score 0.50964, time elapsed 8087.87s
running for params, l2 20, lr 0.4, depth 12 done, cv score 0.50944, time elapsed 8091.21s
running for params, l2 20, lr 0.4, depth 16 done, cv score 0.50916, time elapsed 8094.51s
running for params, l2 20, lr 0.2, depth 8 done, cv score 0.50995, time elapsed 8098.04s
running for params, l2 20, lr 0.2, depth 12 done, cv score 0.50948, time elapsed 8101.56s
running for params, l2 20, lr 0.2, depth 16 done, cv score 0.50941, time elapsed 8105.00s
running for params, l2 20, lr 0.1, depth 8 done, cv score 0.51227, time elapsed 8108.86s
running for params, l2 20, lr 0.1, depth 12 done, cv score 0.51193, time elapsed 8112.67s
running for params, l2 20, lr 0.1, depth 16 done, cv score 0.51178, time elapsed 8116.49s
running for params, l2 1, lr 0.4, depth 8 done, cv score 0.51033, time elapsed 8119.80s
running for params, l2 1, lr 0.4, depth 12 done, cv score 0.51034, time elapsed 8123.04s
running for para